In [3]:
import os
import re
import sys
import glob
import tarfile
import shutil
import pandas as pd
from pathlib import Path


In [66]:
def parse_hcp_session(file_path):
    file_path = hcp_file
    # Build the mapping from rest/task to the bold scan, e.g., bold1, bold2, etc.
    with open(file_path, 'r') as f:
        info = f.read()
    mapping = dict()
    run_lines = info.split('hcpready: true\n')[-1]
    run_list  = run_lines.split('\n')
    run_list  = [x for x in run_list if x != '']
    run_dict  = {x.split(':')[-1]:x.split(':')[0] for x in run_list}
    return run_dict

In [18]:
repo_dir  = '/home/ubuntu/Projects/canbind'
qunex_dir = Path('/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/processed_data/pf-pipelines/qunex-nbridge/studies/CANBIND-20220818-mCcU5pi4')

### Monetary Incentive Delay (MID)

In [13]:
mid_template = Path(repo_dir, 'scripts/mri/task/fsl_templates/MID_template.fsf')
mid_template


PosixPath('/home/ubuntu/Projects/canbind/scripts/mri/task/fsl_templates/MID_template.fsf')

In [134]:
def mid_prepare_task(session_dir, task_template, ev_dir):
    """
    TODO
    """
    sub_id = session_dir.stem.split('_')[0]

    inbox_dir = Path(session_dir, 'inbox')
    os.system(f'sudo chmod 777 {inbox_dir}')

    
    # read scan information 
    hcp_file = Path(session_dir, 'session_hcp.txt')
    hcp_dict = parse_hcp_session(hcp_file)
    if 'bold anhedonia run-01' in list(hcp_dict.keys()):
        print(f'Processing: {session_dir.stem}')
        print(session_dir)
        print(hcp_dict)

        # get the scan number for MID run
        run_num  = hcp_dict['bold anhedonia run-01']

        # create EV directory
        scan_dir = Path(session_dir, f'hcp/{session_dir.parts[-1]}/MNINonLinear/Results/{int(run_num)}')
        ev_dest_dir   = Path(scan_dir, 'EVs')
        if not ev_dest_dir.exists():
            os.system(f'sudo mkdir {ev_dest_dir}')
            os.system(f'sudo chmod 777 {ev_dest_dir}')

        # copy EV files into the session directory
        ev_files = list(Path(ev_dir).glob(f'sub-{sub_id}*'))
        for ev_orig in ev_files:
            ev_dest = Path(ev_dest_dir, ev_orig.parts[-1])
            shutil.copy(ev_orig, ev_dest)
        
        # FSL Parametric Modulation
        tmp_dir = '/Users/kevin.anderson/embarc_prepare_fsf/reward/sub-CU0131_baseline_task-reward_ftype-taskdat_study-EMBARC_ev-Choice.txt'
        replace_dict = {
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/bold2": f'../{run_num}_Atlas',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-CueIncentive.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueNoIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-CueNoIncentive.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-Target.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-Target.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-FbkHitIncentive.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitNonIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-FbkHitNonIncentive.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-FbkMissIncentive.txt',
            "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissNonIncentive.txt": f'../EVs/sub-{sub_id}_ses-01_task-MID_contrast-FbkMissNonIncentive.txt'
        }

        # Create FSF file
        with open(task_template, 'r') as f:
            fsf_lines = f.readlines()

        new_lines = []
        for line in fsf_lines:
            for key in replace_dict.keys():
                if key in line:
                    print(line)
                    line = line.replace(key, replace_dict[key])
            new_lines.append(line)

        # write scan-specific FSF file
        new_fsf = Path(scan_dir, 'MID_hp200_s4_level1.fsf')
        os.system(f'sudo touch {new_fsf}')
        os.system(f'sudo chmod 777 {new_fsf}')
        with open(new_fsf, 'w') as f:
            f.writelines(new_lines)



In [133]:
ev_dir      = '/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/organized_raw_data/mri/fmri_task_timings/MID'
session_dir = Path('/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/processed_data/pf-pipelines/qunex-nbridge/studies/CANBIND-20220818-mCcU5pi4/sessions/TGH0076_01')
mid_prepare_task(session_dir, mid_template, ev_dir)



Processing: TGH0076_01
/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/processed_data/pf-pipelines/qunex-nbridge/studies/CANBIND-20220818-mCcU5pi4/sessions/TGH0076_01
{'T1w': '01', 'bold anhedonia run-01': '02', 'bold gonogo run-01': '03'}
set feat_files(1) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/bold2"

set fmri(custom1) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueIncentive.txt"

set fmri(custom2) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueNoIncentive.txt"

set fmri(custom3) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-Target.txt"

set fmri(custom4) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitIncentive.txt"

set fmri(custom5) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitNonIncentive.txt"

set fmri(custom6) "/Users/kevin.anderson/c

In [73]:
hcp_file = '/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/processed_data/pf-pipelines/qunex-nbridge/studies/CANBIND-20220818-mCcU5pi4/sessions/UBC0034_01'
hcp_file = Path(session_dir, 'session_hcp.txt')
hcp_dict = parse_hcp_session(hcp_file)
if 'bold anhedonia run-01' in list(hcp_dict.keys()):
    print(hcp_file)




/home/ubuntu/canbind-fsx/research/imaging/datasets/CANBIND/processed_data/pf-pipelines/qunex-nbridge/studies/CANBIND-20220818-mCcU5pi4/sessions/TGH0076_01/session_hcp.txt


In [ ]:
            # FSL Parametric Modulation
            tmp_dir = '/Users/kevin.anderson/embarc_prepare_fsf/reward/sub-CU0131_baseline_task-reward_ftype-taskdat_study-EMBARC_ev-Choice.txt'
            replace_dict = {
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/bold2": f'../{run_num}_Atlas',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueIncentive.txt": f'../EVs/{task_csv.stem}_contrast-CueIncentive.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueNoIncentive.txt": f'../EVs/{task_csv.stem}_contrast-CueNoIncentive.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-Target.txt": f'../EVs/{task_csv.stem}_contrast-Target.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitIncentive.txt": f'../EVs/{task_csv.stem}_contrast-FbkHitIncentive.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitNonIncentive.txt": f'../EVs/{task_csv.stem}_contrast-FbkHitNonIncentive.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissIncentive.txt": f'../EVs/{task_csv.stem}_contrast-FbkMissIncentive.txt',
                "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissNonIncentive.txt": f'../EVs/{task_csv.stem}_contrast-FbkMissNonIncentive.txt'
            }

In [ ]:
# FSL Parametric Modulation
tmp_dir = '/Users/kevin.anderson/embarc_prepare_fsf/reward/sub-CU0131_baseline_task-reward_ftype-taskdat_study-EMBARC_ev-Choice.txt'
replace_dict = {
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/bold2": f'../{scan_num}_Atlas',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueNoIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-Target.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitNonIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissNonIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt'
}

In [ ]:
sessions_list = list(qunex_dir.glob('sessions/*_*'))
# iterate over all processed CANBIND subjects
for session_dir in sessions_list:
    print(session_dir)

    # read session hcp file
    hcp_file = Path(session_dir, 'session_hcp.txt')
    if hcp_file.exists():
        hcp_dict = parse_hcp_session(hcp_file)
        print(hcp_dict)
        



In [30]:
parse_session_hcp(hcp_file, ['faces run-01'])

In [ ]:
scan_dir = Path(sessions_dir, session_id, 'hcp', session_id, 'MNINonLinear/Results', scan_num)


In [14]:
# Create FSF file
with open(mid_template, 'r') as f:
    reward_fsf = f.readlines()

In [15]:
# FSL Parametric Modulation
tmp_dir = '/Users/kevin.anderson/embarc_prepare_fsf/reward/sub-CU0131_baseline_task-reward_ftype-taskdat_study-EMBARC_ev-Choice.txt'
replace_dict = {
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/bold2": f'../{scan_num}_Atlas',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-CueNoIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-Target.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkHitNonIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt',
    "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/sub-UBC0039_ses-01_task-MID_contrast-FbkMissNonIncentive.txt": f'../EVs/{task_csv.stem}_ev-Choice.txt'
}

['\n',
 '# FEAT version number\n',
 'set fmri(version) 6.00\n',
 '\n',
 '# Are we in MELODIC?\n',
 'set fmri(inmelodic) 0\n',
 '\n',
 '# Analysis level\n',
 '# 1 : First-level analysis\n',
 '# 2 : Higher-level analysis\n',
 'set fmri(level) 1\n',
 '\n',
 '# Which stages to run\n',
 '# 0 : No first-level analysis (registration and/or group stats only)\n',
 '# 7 : Full first-level analysis\n',
 '# 1 : Pre-processing\n',
 '# 2 : Statistics\n',
 'set fmri(analysis) 7\n',
 '\n',
 '# Use relative filenames\n',
 'set fmri(relative_yn) 0\n',
 '\n',
 '# Balloon help\n',
 'set fmri(help_yn) 0\n',
 '\n',
 '# Run Featwatcher\n',
 'set fmri(featwatcher_yn) 1\n',
 '\n',
 '# Cleanup first-level standard-space images\n',
 'set fmri(sscleanup_yn) 0\n',
 '\n',
 '# Output directory\n',
 'set fmri(outputdir) "/Users/kevin.anderson/canbind_task/UBC0039_01/MID/."\n',
 '\n',
 '# TR(s)\n',
 'set fmri(tr) 2.000000 \n',
 '\n',
 '# Total volumes\n',
 'set fmri(npts) 300\n',
 '\n',
 '# Delete volumes\n',
 'set fm